In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import *

In [12]:
x = np.arange(10, dtype=float)
y = 5 * x + 3

print('x :', x)
print('y :', y)

x : [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
y : [ 3.  8. 13. 18. 23. 28. 33. 38. 43. 48.]


## Define Custom Loss Function

In [36]:
def huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    small_error = tf.square(error) / 2
    big_error = threshold  * (tf.abs(error) - threshold / 2)
    return tf.where(tf.abs(error) <= threshold, small_error, big_error)

In [37]:
model = Sequential([layers.Dense(1, input_shape=(1,))])

model.compile(optimizer='sgd', loss=huber_loss)
model.fit(x, y, epochs=500, verbose=0)
model.predict([10])

2022-01-28 08:33:10.408296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-28 08:33:11.980523: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[53.38612]], dtype=float32)

In [38]:
model.weights

[<tf.Variable 'dense_12/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[5.1035337]], dtype=float32)>,
 <tf.Variable 'dense_12/bias:0' shape=(1,) dtype=float32, numpy=array([2.3507814], dtype=float32)>]

## Add Hyperparameters to Functions

In [39]:
def huber_loss_with_threshold(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        small_error = tf.square(error) / 2
        big_error = threshold  * (tf.abs(error) - threshold / 2)
        return tf.where(tf.abs(error) <= threshold, small_error, big_error)
    return huber_loss

NOTE: Instead of directly passing the parameter in the ```huber_loss``` function, it has been passed to a wrapper function ```huber_loss_with_threshold```. This is because the loss parameter in ```model.compile``` expects a function with only two parameters i.e. *y_true* and *y_pred*. So to add a hyperparameters, the loss function should be enclosed in a wrapper function that accepts them

In [45]:
model = Sequential([layers.Dense(1, input_shape=(1,))])

model.compile(optimizer='sgd', loss=huber_loss_with_threshold(threshold=1))
model.fit(x, y, epochs=500, verbose=0)
model.predict([10])

2022-01-28 08:33:40.047313: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-28 08:33:41.605847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[53.37973]], dtype=float32)

In [41]:
model.weights

[<tf.Variable 'dense_13/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[5.07827]], dtype=float32)>,
 <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([2.5091996], dtype=float32)>]

## Define Custom Loss as a Class

In [51]:
class CustomHuberLoss(losses.Loss):
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        small_error = tf.square(error) / 2
        big_error = self.threshold  * (tf.abs(error) - self.threshold / 2)
        return tf.where(tf.abs(error) <= self.threshold, small_error, big_error)

In [59]:
model = Sequential([layers.Dense(1, input_shape=(1,))])

model.compile(optimizer='sgd', loss=CustomHuberLoss(threshold=1.2))
model.fit(x, y, epochs=500, verbose=0)
model.predict([10])

2022-01-28 09:06:20.460651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-28 09:06:22.047423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array([[53.41405]], dtype=float32)

In [61]:
model.get_weights()

[array([[5.1110225]], dtype=float32), array([2.3038254], dtype=float32)]